Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 138.2 MB/s eta 0:00:00


In [1]:
!pip install transformers torch datasets pyarrow evaluate
!pip install --upgrade accelerate
import pandas as pd
import transformers
from transformers import BatchEncoding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df=pd.read_pickle('gdrive/My Drive/Research/dataset/full_df_dedup.pkl')


In [4]:
class customTokenizer:
    def __init__(self, all_tokens):
        self.all_tokens = all_tokens
        self.token_dict = {}
        self.gen_token_dict()
        self.vocab_size = len(self.token_dict)+1

    def gen_token_dict(self):
        for i,token in enumerate(self.all_tokens):
            self.token_dict[token] = i+1
        
    def get_token(self, token):
        token_type = token[1]
        token_value = token[0]
        if token_type == "String":
            return "<STR>"
        elif token_type == "Number":
            return "<NUM>"
        elif token_type == "RegularExpression":
            return "<REGEX>"
        elif token_type == "Template":
            return "<TEMPLATE>"
        elif token_value not in self.token_dict:
            return "<UNK>"
        else:
            return token_value
    

    def tokenize(self, row, max_length=512):
        tokenized = {}
        tokens = eval(row["tokens"]) # this here for the drop duplicates hack
        annotations = row["annotations"]
        
        if len(tokens) < max_length:
            tokenized["input_ids"] = [self.token_dict[self.get_token(token)] for token in tokens] + [0]*(max_length - len(tokens))
            tokenized["token_type_ids"] = annotations + [0]*(max_length - len(tokens))
            tokenized["attention_mask"] = ([1] * len(tokens)) + [0] * (max_length - len(tokens))
        else:
            tokenized["input_ids"] = [self.token_dict[self.get_token(token)] for token in tokens[:max_length]]
            tokenized["token_type_ids"] = annotations[:max_length]
            tokenized["attention_mask"] = [1] * max_length
        
        tokenized["label"] = row["label"]
        return tokenized


def get_all_tokens_from_df(df):
    result = set()
    all_sequences = df["tokens"]
    for i,tokens in all_sequences.items():
        tokens = eval(tokens) # this here for the drop duplicates hack
        for token in tokens:
            token_type = token[1]
            token_value = token[0]
            if token_type == "String":
                result.add("<STR>")
            elif token_type == "Number":
                result.add("<NUM>")
            elif token_type == "RegularExpression":
                result.add("<REGEX>")
            elif token_type == "Template":
                result.add("<TEMPLATE>")
            else:
                result.add(token_value)
    result.add("<UNK>")
    return list(result)

all_tokens = get_all_tokens_from_df(df)
print(len(all_tokens))
custom_tokenizer = customTokenizer(all_tokens)

74555


In [6]:
processed_data = []

for i in range(len(df)):
    processed_data.append(custom_tokenizer.tokenize(df.iloc[i]))

In [41]:
from sklearn.model_selection import train_test_split
new_df = pd.DataFrame(processed_data)
new_df["label"] = new_df["label"].astype(int)
train_df, valid_test_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

valid_df, test_df = train_test_split(
    valid_test_df,
    test_size=0.5,
    random_state=2022
)

In [42]:
print(train_df.label.value_counts())
print(valid_test_df.label.value_counts())
print(valid_df.label.value_counts())
print(test_df.label.value_counts())

1    1795
0    1138
Name: label, dtype: int64
1    440
0    294
Name: label, dtype: int64
1    220
0    147
Name: label, dtype: int64
1    220
0    147
Name: label, dtype: int64


In [24]:
import pyarrow as pa
from datasets import Dataset


In [45]:
from transformers import BertConfig, BertForSequenceClassification
total_acc = []
total_prec = []
total_recall = []
total_f1 = []
for i in range(5):
  train_df, valid_test_df = train_test_split(
      new_df,
      test_size=0.2,
      random_state=i
  )

  valid_df, test_df = train_test_split(
      valid_test_df,
      test_size=0.5,
      random_state=i
  )
  train_hg = Dataset(pa.Table.from_pandas(train_df))
  valid_hg = Dataset(pa.Table.from_pandas(valid_df))
  test_hg = Dataset(pa.Table.from_pandas(test_df))

  config = BertConfig(custom_tokenizer.vocab_size, hidden_size=300, 
                      num_hidden_layers=2, num_attention_heads=2,
                      num_labels=2)
  model = BertForSequenceClassification(config)

  training_args = TrainingArguments(output_dir="./result")

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_hg,
      eval_dataset=valid_hg
  )
  trainer.train()
  result = trainer.predict(test_hg)
  preds = np.argmax(result.predictions, axis=-1)
  metric = evaluate.load("glue", "mrpc")
  acc = metric.compute(predictions=preds, references=result.label_ids)["accuracy"]
  precision,recall,f1,_ = precision_recall_fscore_support(result.label_ids, preds, average='binary')
  print(f"{precision} {recall} {f1} {acc}")
  total_acc.append(acc)
  total_prec.append(precision)
  total_recall.append(recall)
  total_f1.append(f1)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.570200
1000,0.508000


0.7721518987341772 0.8356164383561644 0.8026315789473685 0.7547683923705722


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.598100
1000,0.512700


...

Step,Training Loss
500,0.601700
1000,0.510500


0.7841409691629956 0.7876106194690266 0.7858719646799117 0.7356948228882834


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.593600
1000,0.515200


0.7581967213114754 0.8043478260869565 0.7805907172995781 0.7166212534059946


In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg
)
trainer.train()
result = trainer.predict(test_hg)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.595800
1000,0.515300


TrainOutput(global_step=1101, training_loss=0.5497558517525349, metrics={'train_runtime': 64.5977, 'train_samples_per_second': 136.212, 'train_steps_per_second': 17.044, 'total_flos': 121892706367488.0, 'train_loss': 0.5497558517525349, 'epoch': 3.0})

In [49]:
print(sum(total_prec)/5, np.std(total_prec))
print(sum(total_recall)/5, np.std(total_recall))
print(sum(total_f1)/5, np.std(total_f1))

0.7586290299651108 0.023557445854701077
0.829861075447036 0.03151869945520277
0.7919971166384506 0.015621240553209913
